In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [4]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [5]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [6]:
res = 0.
i = 0

res = torch.dot(inputs[i], input_query)
print(res)

tensor(0.9544)


In [7]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, input_query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [8]:
# normalize scores 

attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
attn_weights_2_tmp

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [9]:
# Softmax self-implementation

def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_naive(attn_scores_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [10]:
# torch implementation - use this when you can

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

In [11]:
query = inputs[1] # 2nd input token is the query in this example

context_vec_2 = torch.zeros(query.shape) # empty context vector of same shape as input vectors (for second context vector)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i # weighted sum done here

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### Simple self-attention mechanism without trainable weights

In [12]:
# Manual implementaton with for loops

attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [13]:
# matrix multiplication to calculate the above MUCH quicker with special notation

attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [14]:
attn_weights = torch.softmax(attn_scores, dim=1) # weights along the rows (dim 1) add to 1
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [15]:
# use matrix multiplication to find all 6 context vectors

all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

#### In one cell:

In [16]:
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=1)
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## 3.4 Implementing self-attention with trainable weights

### 3.4.1 Computing the attention weights step by step

In [ ]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2 # the user can pick these! Can make it the size you want

In [24]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out)) # torch.nn.Parameter is a wrapper around a tensor to make it trainable
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [ ]:
query_2 = x_2 @ W_query

query_2

# turning the 3-d input tensor into a 2-d query tensor

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [33]:
# Query is reused everywhere (when comparing to every other token). However, keys (and values) are unique when used in relation to other tokens. Basically we need to find the key vector of the inputs for every other token in the context (which is full matrix multiplication instead of a vector dotted with a matrix)

keys = inputs @ W_key
values = inputs @ W_value

keys.shape

torch.Size([6, 2])

In [34]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)

In [35]:
keys_2 = keys[1] 
attn_score_22 = torch.dot(query_2, keys_2)
attn_score_22

tensor(1.8524, grad_fn=<DotBackward0>)

#### Calculating attn *scores*

In [ ]:
# Can compute the attn score for the entire context (from the reference point of the second token) by matrix multiplying the query embedding of the 2nd token with the tensor of the key values for all the tokens in the context.

attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

#### Calculating attn *weights* with softmax

In [45]:
d_k = keys.shape[1]

# normalizing the attention scores with the dimensions of the key matrix

attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1) # row-wise softmax -> we are normalizing the key vectors entirely (dim = 0 treats the first entry of each key as a column vector then normalizes)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

In [46]:
# normalized weights should sum up to 1

sum(attn_weights_2)

tensor(1., grad_fn=<AddBackward0>)

In [ ]:
# context vector is the attn weighted sum of all the value vectors (another matrix multiplication)

context_vec_2 = attn_weights_2 @ values

context_vec_2

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)

In summary, the queries and keys together are used to find the **attention scores** which are then normalized and softmaxxed into **attention weights**. The values are applied to all the attention weights to compute the **context vectors**.

### 3.4.2 Implementing a compact self-attention class

We are trying now to get the context vector for all the other input tokens (previously we did it just for the second token) by reusing the code from 3.4.1 to make a reusable class.

In [50]:
import torch.nn as nn

class SelfAttention_V1(nn.Module):
    
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out)) # torch.nn.Parameter is a wrapper around a tensor to make it trainable
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))   

    def forward(self, x):

        queries = inputs @ W_query
        keys = inputs @ W_key
        values = inputs @ W_value

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[1]**0.5, dim=-1)

        context_vec = attn_weights @ values

        return context_vec
    
torch.manual_seed(123)
sa_v1 = SelfAttention_V1(d_in, d_out)
sa_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

Note in the code above that sa_v1[inputs][1] = [0.3061, 0.8210], which means the class works!

In [52]:
m = torch.nn.Linear(2,3)
m.bias

Parameter containing:
tensor([ 0.2911,  0.5878, -0.0934], requires_grad=True)

In [56]:
import torch.nn as nn

# Self attention with linear layers (better implementation)

class SelfAttention_V2(nn.Module):
    
    def __init__(self, d_in, d_out, qkv_bias=False): # the bias term in the nn.Linear, which we don't need (not using a bias in this nn I guess?)
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias) # torch.nn.Linear is an alternative to nn.Parameter (not sure what the benefit is aside from no explicit torch.rand call needed, slightly better weight initialization under the hood according to Sebastian)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)  

    def forward(self, x):

        queries = self.W_query(inputs) # similar behavior -> we are still doing matrix multiplication
        keys = self.W_key(inputs)
        values = self.W_value(inputs)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[1]**0.5, dim=-1)

        context_vec = attn_weights @ values

        return context_vec
    
torch.manual_seed(789)
sa_v2 = SelfAttention_V2(d_in, d_out)
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

## 3.5 Hiding future words with causal attention

For a predictive model, it doesn't make sense for the model to know future words in the context (relative to the current token being looked at)

### 3.5.1 Applying a causal attention mask